<a href="https://colab.research.google.com/github/andy7809/StackOverFlowAnswers/blob/master/TensorflowDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!echo $'text, tag' >> movie_reviews.csv
!echo $'Good Review, 1' >> movie_reviews.csv
!echo $'Bad Review, 0' >> movie_reviews.csv
!echo $'Really Good Review, 1' >> movie_reviews.csv
!echo $'Really Bad Review, 0' >> movie_reviews.csv
!echo $'Average review, 1' >> movie_reviews.csv
!echo $'Slightly Ok Review, 0' >> movie_reviews.csv

Question 0: How do I load my CSV?

In [ ]:
import tensorflow as tf

# Put your filepath here
filepath = 'movie_reviews.csv'
# Do you need default values?
default_values = ['x', 0.0]
# Does your csv have a header?
header = True

csv_dataset = tf.data.experimental.CsvDataset(filepath, default_values, header=header)

def reshape_dataset(txt, tag):
  txt = tf.reshape(txt, shape=(1,))
  tag = tf.reshape(tag, shape=(1,))
  return txt, tag

csv_dataset = csv_dataset.map(reshape_dataset)
csv_dataset

Tensor("Shape:0", shape=(1,), dtype=int32)


<MapDataset shapes: ((1,), (1,)), types: (tf.string, tf.float32)>

Question 1: How do I access the text?

In [ ]:
# you can iterate through the dataset in a training loop!
for entry in csv_dataset:
    # an entry is a tuple: text in the first position, tag in the second.
    print('text: {}, tag: {}'.format(entry[0], entry[1]))

text: b'Good Review', tag: 0.5
text: b'Bad Review', tag: -0.5
text: b'Really Good Review', tag: 5.0
text: b'Really Bad Review', tag: -5.0
text: b'Average review', tag: 0.0
text: b'Slightly Ok Review', tag: 0.10000000149011612


Question 2: How can I create training and testing datasets?

I revised my original implementation with [this!](https://stackoverflow.com/questions/48213766/split-a-dataset-created-by-tensorflow-dataset-api-in-to-train-and-test)

In [ ]:
training = csv_dataset.take(3)
testing = csv_dataset.skip(3)

print('Training Data:')
for entry in training:
    print('text: {}, tag: {}'.format(entry[0], entry[1]))

print('Testing Data:')
for entry in testing:
    print('text: {}, tag: {}'.format(entry[0], entry[1]))

Question 3: How can I turn my text into a vector?

It already is a vector! You can go ahead and train the data. I got the model from [here](https://www.tensorflow.org/hub/tutorials/tf2_text_classification)

In [ ]:
import tensorflow_hub as hub
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

model.fit(csv_dataset, epochs=5)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_3 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_14 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
6/6 [==============================] - 0s 12ms/step - loss: 1.4550 - accuracy: 0.0000e+00
Epoch 2/5
6/6 [==============================] - 0s 5ms/step - loss: 1.3443 - accuracy: 0.0000e+00
Epoch 3/5
6/6 [==============================] - 0s 4ms/step - loss: 1.2653 - accuracy: 0.0000e+00
Epoch 4/5
6/6 [==============================] - 0s 4ms/step - loss: 1.1837 - a